In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import csv
#import cPickle as pickle
import pywFM
from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.cross_validation import train_test_split
from fastFM.mcmc import FMClassification, FMRegression
#from pyfm import pylibfm

# set path
default_path = "/Users/mayritaspring/Desktop/Github/Data/Recommender System"
import os
os.chdir(default_path)

/Users/mayritaspring/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# 1.Data

(1) Criteo (https://www.kaggle.com/c/criteo-display-ad-challenge/data)
- Label - Target variable that indicates if an ad was clicked (1) or not (0)
- I1-I13 - A total of 13 columns of integer features (mostly count features)
- C1-C26 - A total of 26 columns of categorical features. The values of these features have been hashed onto 32 bits for anonymization purposes

In [2]:
train_data = pd.read_csv('./train_tiny.csv')

In [3]:
train_data.head()

,Id,Label,I1,I2,I3,I4,I5,I6,I7,I8,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,10000000,0,1.0,1,5.0,0.0,1382.0,4.0,15.0,2.0,...,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,NaN,3a171ecb,c5c50484,e8b83407,9727dd16
1,10000001,0,2.0,0,44.0,1.0,102.0,8.0,2.0,2.0,...,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,NaN,3a171ecb,43f13e8b,e8b83407,731c3655
2,10000002,0,2.0,0,1.0,14.0,767.0,89.0,4.0,2.0,...,8efede7f,3412118d,NaN,NaN,e587c466,ad3062eb,3a171ecb,3b183c5c,NaN,NaN
3,10000003,0,NaN,893,NaN,NaN,4392.0,NaN,0.0,0.0,...,1e88c74f,74ef3502,NaN,NaN,6b3a5ca6,NaN,3a171ecb,9117a34a,NaN,NaN
4,10000004,0,3.0,-1,NaN,0.0,2.0,0.0,3.0,0.0,...,1e88c74f,26b3c7a7,NaN,NaN,21c9516a,NaN,32c7478e,b34f3128,NaN,NaN


In [4]:
test_data = pd.read_csv('./test_tiny.csv')

In [5]:
test_data.head()

,Id,Label,I1,I2,I3,I4,I5,I6,I7,I8,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,60000000,0,NaN,29,50.0,5.0,7260.0,437.0,1.0,4.0,...,3486227d,1616f155,21ddcdc9,5840adea,2c277e62,NaN,423fab69,54c91918,9b3e8820,e75c9ae9
1,60000001,0,27.0,17,45.0,28.0,2.0,28.0,27.0,29.0,...,e5ba7672,b4abdd09,21ddcdc9,5840adea,36a7ab86,NaN,32c7478e,85e4d73f,010f6491,ee63dd9b
2,60000002,0,1.0,1,19.0,7.0,1.0,3.0,1.0,7.0,...,d4bb7bd8,bd17c3da,1d04f4a4,a458ea53,82bdc0bb,NaN,32c7478e,5bdcd9c4,010f6491,cca57dcc
3,60000003,0,4.0,1,NaN,6.0,1051.0,134.0,4.0,35.0,...,07c540c4,bdc06043,NaN,NaN,6dfd157c,ad3062eb,423fab69,ef089725,NaN,NaN
4,60000004,0,7.0,1,25.0,10.0,139.0,74.0,48.0,13.0,...,27c07bd6,395856b0,21ddcdc9,a458ea53,9c3eb598,ad3062eb,3a171ecb,c0b8dfd6,001f3601,7a2fb9af


In [6]:
print("Train samples: {}, test samples: {}".format(len(train_data), len(test_data)))

Train samples: 1999, test samples: 1999


In [7]:
# Based on Kaggle kernel by Scirpus
def convert_to_ffm(df,type,numerics,categories,features):
    currentcode = len(numerics)
    catdict = {}
    catcodes = {}
    # Flagging categorical and numerical fields
    for x in numerics:
         catdict[x] = 0
    for x in categories:
         catdict[x] = 1
    
    nrows = df.shape[0]
    ncolumns = len(features)
    with open(str(type) + "_ffm.txt", "w") as text_file:
# Looping over rows to convert each row to libffm format
        for n,r in enumerate(range(nrows)):
            datastring = ""
            datarow = df.iloc[r].to_dict()
            datastring += str(int(datarow['Label']))
             # For numerical fields, we are creating a dummy field here
            for i, x in enumerate(catdict.keys()):
                if(catdict[x]==0):
                    datastring = datastring + " "+str(i)+":"+ str(i)+":"+ str(datarow[x])
                else:
            # For a new field appearing in a training example
                    if(x not in catcodes):
                        catcodes[x] = {}
                        currentcode +=1
                        catcodes[x][datarow[x]] = currentcode #encoding the feature
            # For already encoded fields
                    elif(datarow[x] not in catcodes[x]):
                        currentcode +=1
                        catcodes[x][datarow[x]] = currentcode #encoding the feature
                    code = catcodes[x][datarow[x]]
                    datastring = datastring + " "+str(i)+":"+ str(int(code))+":1"

            datastring += '\n'
            text_file.write(datastring)

(2) Missing Value Handling

In [8]:
#Training Data
num_col_tr = train_data.iloc[:,2:15]
cat_col_tr = train_data.iloc[:,15:41]

num_col_tr = pd.DataFrame(num_col_tr.fillna(num_col_tr.mean()))
cat_col_tr = pd.DataFrame(cat_col_tr.fillna(0))
all_col_tr = pd.concat([num_col_tr,cat_col_tr],axis=1)

print (num_col_tr.shape)
print (cat_col_tr.shape)
print (all_col_tr.shape)

(1999, 13)
(1999, 26)
(1999, 39)


In [9]:
#Testing Data
num_col_te = test_data.iloc[:,2:15]
cat_col_te = test_data.iloc[:,15:41]

num_col_te = pd.DataFrame(num_col_te.fillna(num_col_tr.mean()))
cat_col_te = pd.DataFrame(cat_col_te.fillna(0))
all_col_te = pd.concat([num_col_te,cat_col_te],axis=1)

print (num_col_te.shape)
print (cat_col_te.shape)
print (all_col_te.shape)

(1999, 13)
(1999, 26)
(1999, 39)


In [10]:
train_data_Label = pd.concat([train_data.Label,all_col_tr],axis=1)
convert_to_ffm(train_data_Label,'Train',list(num_col_tr),list(cat_col_tr),list(all_col_tr))

In [11]:
test_data_Label = pd.concat([test_data.Label,all_col_te],axis=1)
convert_to_ffm(test_data_Label,'Test',list(num_col_te),list(cat_col_te),list(all_col_te))

# Packages
## (1) xlearn

In [1]:
import xlearn as xl

ffm_model = xl.create_ffm()
ffm_model.setTrain("./train_tiny.csv")  
ffm_model.setValidate("./test_tiny.csv")
# param:
#  0. task: binary classification
#  1. learning rate : 0.2
#  2. regular lambda : 0.002
param = {'task':'binary', 'lr':0.2, 'lambda':0.002}

In [ ]:
# Train model
ffm_model.fit(param, "./model.out")


In [14]:
#ffm_model.setTest("./test.tiny.csv")
ffm_model.predict("./model.out", "./output.csv")